In [1]:
import pickle
import numpy as np
import sys
import os
import pandas as pd
import csv
import random

In [2]:
path_project = '/beegfs/yw3004/projects/ismir2018_LBD/'

# 1. Read metadata to get artist ids

In [3]:
with open(os.path.join(path_project, 'openmic-20k-metadata.csv')) as csvfile:
    reader = csv.DictReader(csvfile)
    data = [r for r in reader]

In [4]:
data[0];

In [5]:
with open(os.path.join(path_project, 'openmic-20k-metadata.csv')) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    key_artist_dict = {}
    artist_keys_dict = {}

    for row in readCSV:
        artist = row[4]
        key = row[39]
        key_artist_dict[key] = artist
        if artist not in artist_keys_dict.keys():
            artist_keys_dict[artist] = [key]
        else:
            artist_keys_dict[artist].append(key)

In [6]:
key_artist_dict;

In [7]:
artist_keys_dict;

In [8]:
len(artist_keys_dict)

6828

# 2. Read instrument y_dict to neg and pos#

In [9]:
instrument_all = ['accordion', 'banjo', 'bass', 'cello', 'clarinet', 'cymbals', 'drums', 'flute', 'guitar', 
                  'mandolin', 'mallet_percussion', 'organ', 'piano', 'saxophone', 'synthesizer', 'trombone', 
                  'trumpet', 'ukulele', 'violin', 'voice']

In [10]:
for inst in instrument_all:
    with open(os.path.join(path_project, 'inst_labels', inst + '_y.pickle'), 'rb') as f:
        y_dict = pickle.load(f, encoding='latin1')
        print (inst, len(y_dict))

accordion 1500
banjo 2218
bass 1888
cello 1949
clarinet 2385
cymbals 1735
drums 1747
flute 2084
guitar 1650
mandolin 2463
mallet_percussion 1802
organ 1890
piano 1720
saxophone 2365
synthesizer 1602
trombone 1584
trumpet 2916
ukulele 2425
violin 2033
voice 1564


In [11]:
inst = 'guitar'

In [12]:
with open(os.path.join(path_project,'inst_labels',inst+'_y.pickle'), 'rb') as f:
    y_dict = pickle.load(f, encoding='latin1')

In [13]:
neg_keys = []
pos_keys = []

for y in y_dict:
    if y_dict[y] == 0:
        neg_keys.append(y)
    else:
        pos_keys.append(y)

In [14]:
len(pos_keys)

1137

In [15]:
len(neg_keys)

513

# 3. Sort sample keys by artist, split artists for test/val/train

In [23]:
key_sort_by_artist = {}
for key in y_dict.keys():
    artist = key_artist_dict[key]
    if artist not in key_sort_by_artist.keys():
        key_sort_by_artist[artist] = [key]
    else:
        key_sort_by_artist[artist].append(key)

In [33]:
num_artist = len(key_sort_by_artist)
num_artist

1038

In [38]:
with open(os.path.join(path_project, 'training_data', inst + '_key_sort_by_artist.pickle'), 'wb') as f:
    pickle.dump(key_sort_by_artist, f, protocol=pickle.HIGHEST_PROTOCOL) 

In [24]:
artists = list(key_sort_by_artist.keys())
random.shuffle(artists)
num_test_artist = 300
num_val_artist = 200
test_artist = artists[:num_test_artist]
val_artist = artists[num_test_artist:num_test_artist+num_val_artist]
train_artist = artists[num_test_artist+num_val_artist:]

In [25]:
len(train_artist)

538

In [26]:
with open(os.path.join(path_project, 'training_data', inst + '_test_artist.pickle'), 'wb') as f:
    pickle.dump(test_artist, f, protocol=pickle.HIGHEST_PROTOCOL)  
with open(os.path.join(path_project, 'training_data', inst + '_val_artist.pickle'), 'wb') as f:
    pickle.dump(val_artist, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(path_project, 'training_data', inst + '_train_artist.pickle'), 'wb') as f:
    pickle.dump(train_artist, f, protocol=pickle.HIGHEST_PROTOCOL)

# 4. Load features, build X, y for test/val/train

In [39]:
with open(os.path.join(path_project,'mean_feat_vggish_openmic.pickle'), 'rb') as f:
    mean_feat = pickle.load(f, encoding='latin1')
with open(os.path.join(path_project,'std_feat_vggish_openmic.pickle'), 'rb') as f:
    std_feat = pickle.load(f, encoding='latin1')

In [68]:
X_test = np.zeros([1, 256])
y_test = np.zeros(1)
X_val = np.zeros([1, 256])
y_val = np.zeros(1)
X_train = np.zeros([1, 256])
y_train = np.zeros(1)

for artist in test_artist:
    for key in key_sort_by_artist[artist]:
        X_test = np.vstack([X_test, np.concatenate((mean_feat[key], std_feat[key]))])
        y_test = np.vstack([y_test, y_dict[key]])
        
for artist in val_artist:
    for key in key_sort_by_artist[artist]:
        X_val = np.vstack([X_val, np.concatenate((mean_feat[key], std_feat[key]))])
        y_val = np.vstack([y_val, y_dict[key]])

for artist in train_artist:
    for key in key_sort_by_artist[artist]:
        X_train = np.vstack([X_train, np.concatenate((mean_feat[key], std_feat[key]))])
        y_train = np.vstack([y_train, y_dict[key]])
        
X_test = np.delete(X_test, 0, 0)
y_test = np.delete(y_test, 0)
X_val = np.delete(X_val, 0, 0)
y_val = np.delete(y_val, 0)
X_train = np.delete(X_train, 0, 0)
y_train = np.delete(y_train, 0)

In [69]:
mask_test = np.any(np.isnan(X_test), axis=1)
mask_val = np.any(np.isnan(X_val), axis=1)
mask_train = np.any(np.isnan(X_train), axis=1)

X_test = X_test[~mask_test]
y_test = y_test[~mask_test]
X_val = X_val[~mask_val]
y_val = y_val[~mask_val]
X_train = X_train[~mask_train]
y_train = y_train[~mask_train]

In [79]:
X_test.shape

(387, 256)

In [71]:
X_val.shape

(268, 256)

In [72]:
X_train.shape

(731, 256)

In [81]:
with open(os.path.join(path_project, 'training_data', inst + '_X_test.pickle'), 'wb') as f:
    pickle.dump(X_test, f, protocol=pickle.HIGHEST_PROTOCOL) 
with open(os.path.join(path_project, 'training_data', inst +'_y_test.pickle'), 'wb') as f:
    pickle.dump(y_test, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(path_project, 'training_data', inst +'_X_val.pickle'), 'wb') as f:
    pickle.dump(X_val, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(path_project, 'training_data', inst +'_y_val.pickle'), 'wb') as f:
    pickle.dump(y_val, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(path_project, 'training_data', inst +'_X_train.pickle'), 'wb') as f:
    pickle.dump(X_train, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(path_project, 'training_data', inst +'_y_train.pickle'), 'wb') as f:
    pickle.dump(y_train, f, protocol=pickle.HIGHEST_PROTOCOL)

# 5. Build X_pool #

In [34]:
X_pool = np.zeros([1, 256])
key_pool = np.array(['000000_000000'])

for artist in artist_keys_dict:
    if artist != 'artist_id':
        if artist not in test_artist:
            for key in artist_keys_dict[artist]:
                X_pool = np.vstack([X_pool, np.concatenate((mean_feat[key], std_feat[key]))])
                key_pool = np.vstack([key_pool, np.array([key])])
                
X_pool = np.delete(X_pool, 0, 0)
key_pool = np.delete(key_pool, 0, 0)

In [75]:
mask_pool = np.any(np.isnan(X_pool), axis=1)
X_pool = X_pool[~mask_pool]
key_pool = key_pool[~mask_pool]

In [36]:
X_pool.shape

(17927, 256)

In [37]:
with open(os.path.join(path_project, 'training_data', inst +'_X_pool.pickle'), 'wb') as f:
    pickle.dump(X_pool, f, protocol=pickle.HIGHEST_PROTOCOL) 
with open(os.path.join(path_project, 'training_data', inst +'_key_pool.pickle'), 'wb') as f:
    pickle.dump(key_pool, f, protocol=pickle.HIGHEST_PROTOCOL) 